# Exercises II

Let's set up an example scenario as perspective for our regression exercises using the Zillow dataset.

As a Codeup data science graduate, you want to show off your skills to the Zillow data science team in hopes of getting an interview for a position you saw pop up on LinkedIn. You thought it might look impressive to build an end-to-end project in which you use some of their Kaggle data to predict property values using some of their available features; who knows, you might even do some feature engineering to blow them away. Your goal is to predict the values of single unit properties using the observations from 2017.

In these exercises, you will complete the first step toward the above goal: acquire and prepare the necessary Zillow data from the zillow database in the Codeup database server.

## Imports

In [1]:
import numpy as np
import pandas as pd
import os

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

from env import get_db_url
import wrangle as w

## 1. Acquire `bedroomcnt`, `bathroomcnt`, `calculatedfinishedsquarefeet`, `taxvaluedollarcnt`, `yearbuilt`, `taxamount`, and `fips` from the `zillow` database for all 'Single Family Residential' properties.

In [2]:
def check_file_exists(filename, query, url):
    if os.path.exists(filename):
        df = pd.read_csv(filename, index_col=0)
        
    else:
        df = pd.read_sql(query,url)
        df.to_csv(filename)
        
    return df

In [3]:
def acquire_zillow():
    filename = 'zillow.csv'
    url = get_db_url('zillow')
    query = """
           select bedroomcnt, 
                bathroomcnt,
                calculatedfinishedsquarefeet,
                taxvaluedollarcnt,
                yearbuilt,
                taxamount, 
                fips 
            from properties_2017
            where propertylandusetypeid = 261
            """

    df = check_file_exists(filename,query,url)
    return df

In [4]:
df = acquire_zillow()

In [5]:
df

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0
...,...,...,...,...,...,...,...
2152858,4.0,3.0,2262.0,960756.0,2015.0,13494.52,6059.0
2152859,4.0,4.5,3127.0,536061.0,2014.0,6244.16,6059.0
2152860,0.0,0.0,NaN,208057.0,NaN,5783.88,6059.0
2152861,3.0,2.5,1974.0,424353.0,2015.0,5302.70,6059.0


## 2. Using your acquired Zillow data, walk through the summarization and cleaning steps in your `wrangle.ipynb` file like we did above. You may handle the missing values however you feel is appropriate and meaningful; remember to document your process and decisions using markdown and code commenting where helpful.

In [6]:
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 2152863 entries, 0 to 2152862
Data columns (total 7 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   bedroomcnt                    2152852 non-null  float64
 1   bathroomcnt                   2152852 non-null  float64
 2   calculatedfinishedsquarefeet  2144379 non-null  float64
 3   taxvaluedollarcnt             2152370 non-null  float64
 4   yearbuilt                     2143526 non-null  float64
 5   taxamount                     2148421 non-null  float64
 6   fips                          2152863 non-null  float64
dtypes: float64(7)
memory usage: 131.4 MB


In [7]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
bedroomcnt,2152852.0,3.287196,0.954754,0.00,3.00,3.00,4.00,25.00
bathroomcnt,2152852.0,2.230688,0.999280,0.00,2.00,2.00,3.00,32.00
calculatedfinishedsquarefeet,2144379.0,1862.855178,1222.125124,1.00,1257.00,1623.00,2208.00,952576.00
taxvaluedollarcnt,2152370.0,461896.237963,699676.049600,1.00,188170.25,327671.00,534527.00,98428909.00
yearbuilt,2143526.0,1960.949681,22.162196,1801.00,1949.00,1958.00,1976.00,2016.00
taxamount,2148421.0,5634.865978,8178.910249,1.85,2534.98,4108.95,6414.32,1337755.86
fips,2152863.0,6048.377335,20.433292,6037.00,6037.00,6037.00,6059.00,6111.00


In [8]:
for col in df.columns:
    print(col)
    print(df[col].isna().sum())
    print()

bedroomcnt
11

bathroomcnt
11

calculatedfinishedsquarefeet
8484

taxvaluedollarcnt
493

yearbuilt
9337

taxamount
4442

fips
0



In [9]:
df = df.dropna()
df

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0
6,3.0,4.0,1620.0,847770.0,2011.0,10244.94,6037.0
7,3.0,2.0,2077.0,646760.0,1926.0,7924.68,6037.0
11,0.0,0.0,1200.0,5328.0,1972.0,91.60,6037.0
14,0.0,0.0,171.0,6920.0,1973.0,255.17,6037.0
...,...,...,...,...,...,...,...
2152856,4.0,4.0,4375.0,422400.0,2015.0,13877.56,6037.0
2152858,4.0,3.0,2262.0,960756.0,2015.0,13494.52,6059.0
2152859,4.0,4.5,3127.0,536061.0,2014.0,6244.16,6059.0
2152861,3.0,2.5,1974.0,424353.0,2015.0,5302.70,6059.0


In [10]:
for col in df.columns:
    print(col)
    print(df[col].isna().sum())
    print()

bedroomcnt
0

bathroomcnt
0

calculatedfinishedsquarefeet
0

taxvaluedollarcnt
0

yearbuilt
0

taxamount
0

fips
0



In [11]:
df = df.rename(columns={'bedroomcnt': 'bed_count', 
                       'bathroomcnt' : 'bath_count',
                       'calculatedfinishedsquarefeet' : 'sq_ft',
                       'taxvaluedollarcnt': 'property_value',
                       'yearbuilt': 'year_built',
                       'taxamount': 'tax_amount',
                       'fips': 'county'})
df

,bed_count,bath_count,sq_ft,property_value,year_built,tax_amount,county
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0
6,3.0,4.0,1620.0,847770.0,2011.0,10244.94,6037.0
7,3.0,2.0,2077.0,646760.0,1926.0,7924.68,6037.0
11,0.0,0.0,1200.0,5328.0,1972.0,91.60,6037.0
14,0.0,0.0,171.0,6920.0,1973.0,255.17,6037.0
...,...,...,...,...,...,...,...
2152856,4.0,4.0,4375.0,422400.0,2015.0,13877.56,6037.0
2152858,4.0,3.0,2262.0,960756.0,2015.0,13494.52,6059.0
2152859,4.0,4.5,3127.0,536061.0,2014.0,6244.16,6059.0
2152861,3.0,2.5,1974.0,424353.0,2015.0,5302.70,6059.0


In [12]:
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 2140235 entries, 4 to 2152862
Data columns (total 7 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   bed_count       2140235 non-null  float64
 1   bath_count      2140235 non-null  float64
 2   sq_ft           2140235 non-null  float64
 3   property_value  2140235 non-null  float64
 4   year_built      2140235 non-null  float64
 5   tax_amount      2140235 non-null  float64
 6   county          2140235 non-null  float64
dtypes: float64(7)
memory usage: 130.6 MB


In [13]:
make_int = []
for col in df.columns:
    has_non_zero_decimal = df[col].apply(lambda x: x % 1 != 0)
    if has_non_zero_decimal.sum() == 0:
        make_int.append(col)
    else:
        continue

In [14]:
for col in make_int:
    df[col] = df[col].astype(int)

In [15]:
df.dtypes

bed_count           int64
bath_count        float64
sq_ft               int64
property_value      int64
year_built          int64
tax_amount        float64
county              int64
dtype: object

In [16]:
df.county = df.county.map({6037:'Los Angeles',6059:'Orange',6111:'Ventura'})
df

,bed_count,bath_count,sq_ft,property_value,year_built,tax_amount,county
4,4,2.0,3633,296425,2005,6941.39,Los Angeles
6,3,4.0,1620,847770,2011,10244.94,Los Angeles
7,3,2.0,2077,646760,1926,7924.68,Los Angeles
11,0,0.0,1200,5328,1972,91.60,Los Angeles
14,0,0.0,171,6920,1973,255.17,Los Angeles
...,...,...,...,...,...,...,...
2152856,4,4.0,4375,422400,2015,13877.56,Los Angeles
2152858,4,3.0,2262,960756,2015,13494.52,Orange
2152859,4,4.5,3127,536061,2014,6244.16,Orange
2152861,3,2.5,1974,424353,2015,5302.70,Orange


In [20]:
for col in df.columns:
    print(df[col].value_counts().sort_index())
    print()

bed_count
0       4397
1      22895
2     334221
3     962944
4     633608
5     150671
6      25117
7       4792
8       1103
9        290
10       118
11        34
12        12
13        15
14         7
15         5
16         2
18         3
25         1
Name: count, dtype: int64

bath_count
0.00       4274
0.50         16
1.00     412582
1.50      31157
1.75          3
2.00     942463
2.50     142827
3.00     422398
3.50      28464
4.00      82039
4.50      19474
5.00      28306
5.50       6201
6.00      10717
6.50       1330
7.00       4381
7.50        382
8.00       1681
8.50        108
9.00        707
9.50         50
10.00       322
10.50        14
11.00       145
11.50         3
12.00        73
12.50         3
13.00        39
14.00        25
14.50         1
15.00        17
16.00        12
17.00         4
18.00         8
19.00         1
19.50         1
20.00         6
32.00         1
Name: count, dtype: int64

sq_ft
1         6
2         2
3         7
7         1
10        2
    

In [17]:
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 2140235 entries, 4 to 2152862
Data columns (total 7 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   bedroomcnt                    2140235 non-null  float64
 1   bathroomcnt                   2140235 non-null  float64
 2   calculatedfinishedsquarefeet  2140235 non-null  float64
 3   taxvaluedollarcnt             2140235 non-null  float64
 4   yearbuilt                     2140235 non-null  float64
 5   taxamount                     2140235 non-null  float64
 6   fips                          2140235 non-null  float64
dtypes: float64(7)
memory usage: 130.6 MB


In [18]:
df = df.rename(columns={'bedroomcnt': 'bed_count', 
                       'bathroomcnt' : 'bath_count',
                       'calculatedfinishedsquarefeet' : 'sq_ft',
                       'taxvaluedollarcnt': 'property_value',
                       'yearbuilt': 'year_built',
                       'taxamount': 'tax_amount'})

In [19]:
df

,bed_count,bath_count,sq_ft,property_value,year_built,tax_amount,fips
4,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0
6,3.0,4.0,1620.0,847770.0,2011.0,10244.94,6037.0
7,3.0,2.0,2077.0,646760.0,1926.0,7924.68,6037.0
11,0.0,0.0,1200.0,5328.0,1972.0,91.60,6037.0
14,0.0,0.0,171.0,6920.0,1973.0,255.17,6037.0
...,...,...,...,...,...,...,...
2152856,4.0,4.0,4375.0,422400.0,2015.0,13877.56,6037.0
2152858,4.0,3.0,2262.0,960756.0,2015.0,13494.52,6059.0
2152859,4.0,4.5,3127.0,536061.0,2014.0,6244.16,6059.0
2152861,3.0,2.5,1974.0,424353.0,2015.0,5302.70,6059.0


In [20]:
df = df.reset_index().drop(columns='index')

In [21]:
df

,bed_count,bath_count,sq_ft,property_value,year_built,tax_amount,fips
0,4.0,2.0,3633.0,296425.0,2005.0,6941.39,6037.0
1,3.0,4.0,1620.0,847770.0,2011.0,10244.94,6037.0
2,3.0,2.0,2077.0,646760.0,1926.0,7924.68,6037.0
3,0.0,0.0,1200.0,5328.0,1972.0,91.60,6037.0
4,0.0,0.0,171.0,6920.0,1973.0,255.17,6037.0
...,...,...,...,...,...,...,...
2140230,4.0,4.0,4375.0,422400.0,2015.0,13877.56,6037.0
2140231,4.0,3.0,2262.0,960756.0,2015.0,13494.52,6059.0
2140232,4.0,4.5,3127.0,536061.0,2014.0,6244.16,6059.0
2140233,3.0,2.5,1974.0,424353.0,2015.0,5302.70,6059.0


In [17]:
def prep_zillow(df):
    df = df.dropna()
    df = df.rename(columns={'bedroomcnt': 'bed_count', 
                           'bathroomcnt' : 'bath_count',
                           'calculatedfinishedsquarefeet' : 'area',
                           'taxvaluedollarcnt': 'property_value',
                           'yearbuilt': 'year_built',
                           'taxamount': 'tax_amount',
                           'fips': 'county'})
    
    make_int = []
    for col in df.columns:
        has_non_zero_decimal = df[col].apply(lambda x: x % 1 != 0)
        if has_non_zero_decimal.sum() == 0:
            make_int.append(col)
        else:
            continue
        
    for col in make_int:
        df[col] = df[col].astype(int)
        
    df.county = df.county.map({6037:'Los Angeles',6059:'Orange',6111:'Ventura'})
    
    df = df.reset_index().drop(columns='index')
    
    return df

In [18]:
prep_zillow(acquire_zillow())

,bed_count,bath_count,area,property_value,year_built,tax_amount,county
0,4,2.0,3633,296425,2005,6941.39,Los Angeles
1,3,4.0,1620,847770,2011,10244.94,Los Angeles
2,3,2.0,2077,646760,1926,7924.68,Los Angeles
3,0,0.0,1200,5328,1972,91.60,Los Angeles
4,0,0.0,171,6920,1973,255.17,Los Angeles
...,...,...,...,...,...,...,...
2140230,4,4.0,4375,422400,2015,13877.56,Los Angeles
2140231,4,3.0,2262,960756,2015,13494.52,Orange
2140232,4,4.5,3127,536061,2014,6244.16,Orange
2140233,3,2.5,1974,424353,2015,5302.70,Orange


## 3. Write a function to split your data into train, validate, and test.

In [19]:
def split_data(dataframe):
    train, validate_test = train_test_split(dataframe, 
                                            train_size=.6, 
                                            random_state=913
                                           )
    validate, test = train_test_split(validate_test,
                                      test_size=0.50, 
                                      random_state=913
                                     )
    return train, validate, test

In [20]:
train, validate, test = split_data(prep_zillow(acquire_zillow()))

In [21]:
train.shape,validate.shape,test.shape

((1284141, 7), (428047, 7), (428047, 7))

## 4. Store all of the necessary functions to automate your process from acquiring the data to returning a cleaned dataframe with no missing values in your `wrangle.py` file. Name your final function `wrangle_zillow`.

In [22]:
train,validate,test = w.wrangle_zillow()

In [23]:
train.shape,validate.shape,test.shape

((1284141, 7), (428047, 7), (428047, 7))

In [24]:
train

,bed_count,bath_count,sq_ft,property_value,year_built,tax_amount,county
993671,4,3.0,1956,293741,1955,3554.38,Orange
1191413,3,2.0,1929,438685,1929,5486.56,Los Angeles
1131915,3,2.0,1188,61191,1952,1073.84,Orange
1998279,2,1.0,1133,44967,1950,847.60,Orange
1417050,4,4.0,3033,423764,2011,5534.04,Los Angeles
...,...,...,...,...,...,...,...
1928393,4,3.0,1976,483594,1970,6546.68,Los Angeles
516098,3,2.0,1988,212339,1965,3226.18,Los Angeles
413602,4,2.0,1773,339172,1977,4046.07,Los Angeles
1010217,3,2.0,1496,443826,1977,5558.30,Los Angeles
